# Different ways to make a query on mongoDB on ***C#***
the idea is create the same query using different techniques and try understanding the benefits of each one. 


## Setup 
___
***Reference***
* [mongo driver doc](https://docs.mongodb.com/drivers/csharp) 

### Dependencies

#### Mongo DB - DOCKER

In [1]:
#!pwsh
docker run -d -p 27017-27017:27017-27017 --name mongo-test mongo 

#### Nuget packages

In [1]:
#r "nuget:MongoDB.Driver"

Installed package MongoDB.Driver version 2.11.2

#### Client 

In [1]:

using MongoDB.Driver;

var client = new MongoClient("mongodb://localhost:27017");
var database = client.GetDatabase("firstDatabase");

In [1]:
public void PrintResult(List<Collection> results){
    Console.WriteLine($"Result Count: {results.Count()}");
    results.ForEach(res => Console.Write($"FirstProperty: {res.FirstProperty}, SecondProperty: {res.SecondProperty}"));
}

### Creating Models

In [1]:
using MongoDB.Bson.Serialization.Attributes;

[BsonIgnoreExtraElements]
public class Collection {
    public string FirstProperty { get; set; }
    public int SecondProperty { get; set; }
}


### Seeding

In [1]:

var collection = database.GetCollection<Collection>("firstCollection");

var collections = new List<Collection>{
    new Collection {FirstProperty = "First", SecondProperty = 1 },
    new Collection {FirstProperty = "Second", SecondProperty = 2 },
    new Collection {FirstProperty = "Third", SecondProperty = 3 },
    new Collection {FirstProperty = "Fourth", SecondProperty = 4 }
};

collection.InsertMany(collections);

### Utils methods

## Queries
___
Explaing the query: 
* The collections will filtered by property ***'FirstProperty'*** not equals ***'First'*** and the property 'SecondProperty' in 1,2 and 4.

### Using ***Filters*** Builder 
***Reference:***
* [Mongo DB Blog](https://www.mongodb.com/blog/post/quick-start-c-and-mongodb--read-operations)

In [1]:
var numbers = new List<int>{1,4,2};

var collectionInFilter = Builders<Collection>.Filter.In(collect => collect.SecondProperty,numbers);
var collectionEqFilter = Builders<Collection>.Filter.Eq(collect => collect.FirstProperty,"First");
var collectionNotInFilter = Builders<Collection>.Filter.Not(collectionEqFilter);
var collectionAndFilter = Builders<Collection>.Filter.And(collectionInFilter,collectionNotInFilter);

var findFilterResult = collection.FindSync(collectionAndFilter).ToList();

PrintResult(findFilterResult);

### Using fluent ***Mongo driver*** Linq 
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)

In [1]:
var numbers = new List<int>{1,4,2};

var fluentResult = collection.AsQueryable()
    .Where(collect => 
        !collect.FirstProperty.Equals("First")
        && numbers.Contains(collect.SecondProperty))
    .ToList();

PrintResult(fluentResult);


### Using DLS ***Mongo driver*** Linq 
***Reference***
* [Mongo driver Doc LINQ](http://mongodb.github.io/mongo-csharp-driver/2.7/reference/driver/crud/linq/)

In [1]:
var numbers = new List<int>{1,4,2};

var collectionQuerable = collection.AsQueryable();

var dlsResult = from collect in collectionQuerable
                where !collect.FirstProperty.Equals("First")
                     && numbers.Contains(collect.SecondProperty)
                select collect;

PrintResult(dlsResult.ToList());